# Index

**Elasticsearch中的index可以理解为类似于数据库中的表的概念，但是两者并不完全相同。**

1. 在`Elasticsearch`中，`Index`是指一个包含了相关文档的逻辑容器。这些文档具有相同的数据结构，例如相同的字段和数据类型。每个文档都有一个唯一的`ID`来标识它。`Elasticsearch`使用`index`来组织、存储和检索文档数据。
2. 与数据库中的表不同，Elasticsearch中的index可以跨越多个物理节点进行分布式存储和处理。此外，Elasticsearch中的index还具有高可用性、易于扩展和自动分片等特性，这些特性使得Elasticsearch非常适合处理大规模数据和高并发访问的场景。
3. 另外，Elasticsearch中的index也可以看作是一种搜索引擎的概念，每个index可以包含多个document，每个document有多个field，可以使用Elasticsearch提供的搜索API进行全文搜索、聚合、过滤等操作。因此，Elasticsearch中的index不仅仅是一个数据容器，更是一个强大的搜索引擎。

在创建`Index`时，需要指定`Index`的名称，并且可以定义一些`Index`的配置参数，例如分片数、副本数、分词器等等。创建`Index`后，可以向其中添加文档，也可以对其中的文档进行搜索、更新或删除操作。

**`Elasticsearch`的`Index`具有以下特点：**

1. 可以包含多个类型的文档，但是在7.0版本中已经移除了多类型的支持。
1. 可以包含多个分片和副本，以实现水平扩展和高可用性。
1. 可以定义`Mappings`来指定文档结构和字段类型。
1. 可以使用不同的分词器对文档进行分词。
1. 可以使用复杂的查询语句进行搜索，例如布尔查询、范围查询、模糊查询等等。
1. 可以使用聚合来对搜索结果进行统计和分组。

**在`Elasticsearch`中，Index是非常重要的概念，它直接决定了数据的组织方式和查询方式。正确地设计和使用Index可以大大提高搜索性能和用户体验。**

# `Python-ElasticSearch`

    ```python
    !pip install elasticsearch
    !pip install elasticsearch-dsl
    ```

# 操作`Index`

In [ ]:
from elasticsearch_dsl import connections, Index

# 连接到 Elasticsearch
connection = connections.create_connection(hosts=["localhost"])

In [ ]:
index = Index('books')

## `.create()`

In [ ]:
index.create()

> 打开: [http://localhost:9200/books/](http://localhost:9200/books/)，显示以下内容：

```json
        {
            books: {
            aliases: { },
            mappings: { },
            settings: {
                index: {
                routing: {
                allocation: {
                include: {
                        _tier_preference: "data_content"
                    }
                }
            },
            number_of_shards: "1",
            provided_name: "books",
            creation_date: "1686734038157",
            number_of_replicas: "1",
            uuid: "llCS-mrfRbu8TMuBMVdqcg",
            version: {
                    created: "8070099"
                        }
                    }
                }
            }
        }
```

## `.exists()`

In [ ]:
# 查询index是否存在
index.exists()

## `.delete()`

In [ ]:
# 删除index
index.delete()

## 查看参数

### 字段信息

In [ ]:
index.get_mapping()

### 设置信息

In [ ]:
index.get_settings()

# 设定`Index`参数

## elasticsearch

In [ ]:
from elasticsearch import Elasticsearch

# 创建 Elasticsearch 客户端
host = 'http://localhost:9200'
es = Elasticsearch(host, verify_certs=False)

In [ ]:
# 定义要创建的索引名称
index_name = "ik_index"

if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

### settings

In [ ]:
# 定义要应用于索引的设置
my_settings = {
    "analysis": {
        "analyzer": {
            "ik_smart": {
                "tokenizer": "ik_smart"
            }
        },
        "search_analyzer": {
            "ik_smart": {
                "tokenizer": "ik_smart"
            }
        },
    }
}
# 创建索引，并应用设置
es.indices.create(index=index_name, settings=my_settings)

In [ ]:
# 查看设定的settings
es.indices.get_settings(index=index_name)

### `.indices.put_mapping`

In [ ]:
# 定义新的映射
new_mapping = {
    "properties": {
        "title": {
            "type": "text",
            "analyzer": "standard"
        },
        "content": {
            "type": "text",
            "analyzer": "english"
        }
    }
}
es.indices.put_mapping(index=index_name, body=new_mapping)

In [ ]:
# 查看映射
es.indices.get_mapping(index=index_name)

## elasticsearch_dsl

### 通过`.settings`设置

In [ ]:
from elasticsearch_dsl import Index, analyzer, tokenizer, connections

# 连接到 Elasticsearch
connection = connections.create_connection(hosts=["localhost"])

In [ ]:
# 定义分析器和分词器
tokenizer_ik = tokenizer('ik_smart')
analyzer_ik = analyzer('ik_smart', tokenizer=tokenizer_ik)

# 定义索引
index_name = 'ik_index'
ik_index = Index(index_name)

# 如果存在则删除已经创建的索引
if Index(index_name).exists():
    Index(index_name).delete()

# 定义设置
my_settings = {
    "analysis": {
        "analyzer": {
            "ik_smart": {
                "tokenizer": "ik_smart"
            }
        },
        "search_analyzer": {
            "ik_smart": {
                "tokenizer": "ik_smart"
            }
        },
    }
}
# 设置索引设置
ik_index.settings(**my_settings)

# 创建索引
ik_index.create()

In [ ]:
# 查看设定的settings
ik_index.get_settings()

#### 设定`analyzer`

In [ ]:
# 指定要创建的索引名称
index_name = "books_ik"

if Index(index_name).exists():
    Index(index_name).delete()

# 定义分析器和分词器
ik_tokenizer = tokenizer('ik_smart')
ik_analyzer = analyzer('ik_smart', tokenizer=ik_tokenizer)

# 创建索引
ik_index = Index(index_name)
ik_index.analyzer(ik_analyzer)
ik_index.create()

In [ ]:
ik_index.get_settings()

### `Document` 中创建 `mapping`

In [ ]:
from elasticsearch_dsl import Document, Index, Text, analyzer

# 定义索引
index_name = 'ik_index'
ik_index = Index(index_name)

@ik_index.document
class MyDocument(Document):
    """在这里设定mappings"""
    title = Text(analyzer=analyzer("ik_max_word"))
    content = Text(analyzer=analyzer("ik_max_word"))
    class Index:
        name = index_name
        
# 如果存在则删除已经创建的索引
if Index(index_name).exists():
    Index(index_name).delete()

ik_index.create()

In [ ]:
# 查看映射
ik_index.get_mapping()

# 定义分词器

In [ ]:
from elasticsearch_dsl import Index, analyzer, tokenizer

In [ ]:
index_name = 'books'

# 定义分析器和分词器
tokenizer_ik = tokenizer('ik_smart')
analyzer_ik = analyzer('ik_smart', tokenizer=tokenizer_ik)

# 创建索引
index = Index(index_name)
index.analyzer(analyzer_ik)

# 如果存在则删除已经创建的索引
if Index(index_name).exists():
    Index(index_name).delete()
    
# 创建索引
index.create()

In [ ]:
chinese_field_config = {
            "type": "string",
            "store": "no",
            "term_vector": "with_positions_offsets",
            "analyzer": "ik_max_word",
            "search_analyzer": "ik_max_word",
            "include_in_all": "true",
            "boost": 8
        }
new_mapping = {
    "properties": {
        "content": {
            "type": "text",
#             "store": "no",
            "term_vector": "with_positions_offsets",
            "analyzer": "ik_smart",
            "search_analyzer": "ik_smart",
        }
    }
}

# index.index
index.put_mapping(body=new_mapping)

In [ ]:
setting = {"analysis": {
    "analyzer": {
        "ik_smart": {
            "type": "custom",
            "tokenizer": "ik_smart"
        }
    }
}}

index.put_settings(body=setting)

In [ ]:
index.get_settings()

In [ ]:
# -*- coding: utf-8 -*-

import elasticsearch


class ElasticSearchClient(object):
    @staticmethod
    def get_es_servers():
        es_servers = [{
            "host": "localhost",
            "port": "9200"
        }]
        es_client = elasticsearch.Elasticsearch(hosts=es_servers)
        return es_client


class LoadElasticSearch(object):
    def __init__(self):
        self.index = "hz"
        self.doc_type = "text"
        self.es_client = ElasticSearchClient.get_es_servers()
        self.set_mapping()

    def set_mapping(self):
        """
        设置mapping
        """
        chinese_field_config = {
            "type": "string",
            "store": "no",
            "term_vector": "with_positions_offsets",
            "analyzer": "ik_max_word",
            "search_analyzer": "ik_max_word",
            "include_in_all": "true",
            "boost": 8
        }

        mapping = {
            self.doc_type: {
                "_all": {"enabled": False},

                "properties": {
                    "document_id": {
                        "type": "integer"
                    },
                    "content": chinese_field_config
                }
            }
        }

        if not self.es_client.indices.exists(index=self.index):
            # 创建Index和mapping
            self.es_client.indices.create(index=self.index, ignore=400)
            self.es_client.indices.put_mapping(index=self.index, doc_type=self.doc_type, body=mapping)

    def add_date(self, row_obj):
        """
        单条插入ES
        """
        _id = row_obj.get("_id", 1)
        row_obj.pop("_id")
        self.es_client.index(index=self.index, doc_type=self.doc_type, body=row_obj, id=_id)


if __name__ == '__main__':

    content_ls = [
        u"美国留给伊拉克的是个烂摊子吗",
        u"公安部：各地校车将享最高路权",
        u"中韩渔警冲突调查：韩警平均每天扣1艘中国渔船",
        u"中国驻洛杉矶领事馆遭亚裔男子枪击 嫌犯已自首"
    ]

    load_es = LoadElasticSearch()
    # 插入单条数据测试
    for index, content in enumerate(content_ls):
        write_obj = {
            "_id": index,
            "document_id": index,
            "content": content
        }
        load_es.add_date(write_obj)


> *其他功能后续再补充*

-----